In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.optimize as opt

Large sphere mass = 0.8807 ± 0.0001 kg

Large sphere radius = 0.0268 ± 0.0001 m

Small sphere mass = 0.01548 ± 0.00001 kg

Small sphere radius = 0.00684 ± 0.00001 m

Distance between small and large sphere = 0.048 ± 0.005 m

Distance from center of beam to center of small spheres = 0.0662 ± 0.0001 m

Beam length = 0.1483 ± 0.0005 m

Beam width = 0.0121 ± 0.0005 m

Beam mass = 0.0091 ± 0.0001 kg

In [23]:
def get_x(angles, theta_uncert):
    '''Calculates x, defined as e^{-bT/2} based on a series of turning point angles for the oscillator in free decay.
    Parameters - 
    angles: an array of consecutive turning points, given in mrad. Assumes an odd number of elements.
    theta_uncert: uncertainty in the angle of each turning point, in mrad, assumed to always be the same
    Returns - 
    (x, uncert): a tuple of x and its uncertainty
    '''
    angles = angles / 1000.
    theta_uncert = theta_uncert / 1000.
    
    x1 = 1. - ((angles[0] - angles[-1]) / (np.sum(angles[0::2]) - np.sum(angles[1::2])))
    x2 = 1. - (((angles[1:-1])[0] - (angles[1:-1])[-1]) / (np.sum((angles[1:-1])[0::2]) - np.sum((angles[1:-1])[1::2])))
    x = (x1 + x2) / 2.
    
    x_uncert = theta_uncert * (1-x) * np.sqrt((len(angles) - 1) * (1-x)**2 + 2*x) / (np.abs(angles[0] - angles[-1]))
    return (x, x_uncert)

In [24]:
free_angles = np.array([56.12, 8.0, 47.23, 15.58, 41.93, 20.48, 37.94, 23.70, 36.03, 26.47, 35.19])
(x, x_unc) = get_x(free_angles, 0.1);
print(x, "±", x_unc)

0.8006446500868916 ± 0.0013465859746593367


In [25]:
def get_theta_d(angles, x, theta_uncert, x_uncert):
    '''Finds the displacement of equilibrium from that of free decay when the large masses are in place.
    Parameters - 
    angles: an array of consecutive turning points when the oscillator is being driven, in mrad. Assumes odd number of elements.
    x: determined by get_x, using free oscillations
    theta_uncert: uncertainty in the angle of each turning point, in mrad, assumed to always be the same
    x_uncert: uncertainty in x in mrad, also determined by get_x
    Returns - 
    (theta_d, uncert): a tuple of the displacement and its uncertainty
    '''
    angles = angles / 1000.
    theta_uncert = theta_uncert / 1000.
    x_uncert = x_uncert / 1000.
    
    theta_d = ((1-x)*(np.sum(angles[0::2]) - np.sum(angles[1::2])) - angles[0] + x*angles[-1]) / ((len(angles) - 1) * (1 + x))
    
    uncert1 = theta_uncert * np.sqrt((len(angles) - 1) * (1-x)**2 + 2*x) / ((len(angles) - 1) * (1+x))
    uncert2 = x_uncert * (2*(np.sum(angles[0::2]) - np.sum(angles[1::2]) - angles[-1]) + (angles[-1] - angles[0])) / ((len(angles) - 1) * (1 + x)**2)
    uncert = np.sqrt(uncert1**2 + uncert2**2)
    return (theta_d, uncert)

In [26]:
def get_K(period, m, r, d, mb, lb, wb):
    '''
    period: period of oscillation in seconds
    m: mass of each small sphere, assumed to be both equal
    r: radius of each small sphere, in meters
    d: distance from center of beam to each small sphere, in meters
    mb: mass of beam
    lb: length of beam
    wb: width of beam
    '''
    
    m_unc = 0.0001
    d_unc = 0.001
    r_unc = 0.00001
    T_unc = 10
    
    Is = 2*(m*d**2 + (2./5.)*m*r**2)
    Ib = mb*(lb**2 + wb**2) / 12.1
    I = Is + Ib
    K = ((4*np.pi**2) / period**2) * I
    
    I_unc_1 = m_unc * (2*d**2 + (4./5)*r**2)
    I_unc_2 = d_unc * 4*m*d
    I_unc_3 = r_unc * (8./5)*m*r
    
    I_unc = np.sqrt(I_unc_1**2 + I_unc_2**2 + I_unc_3**2)
    
    K_unc = ((4*np.pi**2) / (period**2)) * np.sqrt(I_unc**2 + (4 * I**2 * T_unc**2)/(period**2))
    
    return (K, K_unc)

In [27]:
def get_I(period, m, r, d, mb, lb, wb):
    '''
    period: period of oscillation in seconds
    m: mass of each small sphere, assumed to be both equal
    r: radius of each small sphere, in meters
    d: distance from center of beam to each small sphere, in meters
    mb: mass of beam
    lb: length of beam
    wb: width of beam
    '''
    
    Is = 2*(m*d**2 + (2./5.)*m*r**2)
    Ib = mb*(lb**2 + wb**2) / 12.
    I = Is + Ib
    return I

In [81]:
def get_G(K, theta_d, R, M, m, d, K_unc, theta_d_unc): #This does not have the corrections
    '''
    K: found from get_K
    theta_d: found from get_theta_d
    R: distance between the centers of the large and small spheres
    M: mass of each large sphere
    m: mass of each small sphere
    d: distance from center of beam to each small sphere
    '''
    
    R_unc = 0.005
    M_unc = 0.0001
    m_unc = 0.00001
    d_unc = 0.001
    
    G = (K * theta_d * R**2) / (2 * M * m * d)
    
    G_unc = ((R) / (M*m*d)) * np.sqrt( (theta_d**2 * R**2 * K_unc**2)/(4) + (K**2 * R**2 * theta_d_unc**2)/(4) + (K**2 * theta_d**2 * R_unc**2)/(1) + (K**2 * theta_d**2 * R**2 * M_unc**2)/(4*M**2) + (K**2 * theta_d**2 * R**2 * m_unc**2)/(4*m**2) + (K**2 * theta_d**2 * R**2 * d_unc**2)/(4*d**2))
    
    return (G, G_unc)

In [82]:
K, K_unc = get_K(260, 0.01548, 0.00684, 0.0662, 0.0091, 0.1483, 0.01205);
print(K, "±", K_unc)

8.929892181196249e-08 ± 7.2924684736530944e-09


In [104]:
driven_angles = np.array([47.11, 17.52, 46.68, 20.21, 45.56])

theta_d, theta_d_unc = get_theta_d(driven_angles, x, 0.1, x_unc)
print(theta_d, "±", theta_d_unc)

0.0013364464937132877 ± 1.8420458213445837e-05


In [105]:
G, G_unc = get_G(K, theta_d, 0.048, 0.8807, 0.01548, 0.0662, K_unc, theta_d_unc)
print(G, "±", G_unc)

1.523327660898296e-10 ± 3.4229245442399066e-11


In [99]:
driven_angles2 = np.array([25.08, -11.44, 24.39, -10.94, 22.70, -8.88, 20.14, -6.93, 18.63, -5.49, 16.80])
theta_d2, theta_d2_unc = get_theta_d(driven_angles2, x, 0.1, x_unc)
print(theta_d2, "±", theta_d2_unc)

0.0012520140607685665 ± 7.851415400852486e-06


In [100]:
G2, G2_unc = get_G(K, theta_d2, 0.048, 0.8807, 0.01548, 0.0662, K_unc, theta_d2_unc)
print(G2, "±", G2_unc)

1.4270886710197926e-10 ± 3.2018875632197715e-11


In [101]:
G_corrected = G*0.9571
G2_corrected = G2*0.9571
print(G_corrected, "±", G_unc)
print(G2_corrected, "±", G2_unc)

1.457976904245759e-10 ± 3.4229245442399066e-11
1.3658665670330434e-10 ± 3.2018875632197715e-11


In [109]:
print(0.5 * (G_corrected + G2_corrected), "±", 0.5 * (G_unc + G2_unc))

1.4119217356394014e-10 ± 3.312406053729839e-11
